In [ ]:
#Imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import os
from config import *
import json
import time
import ast
import numpy as np

In [ ]:
api_key = "YHJOJUT5YAKRNBR5"

In [ ]:
def _get_data(symbols,time_from,time_to,api_key):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbols}&time_from={time_from}&topics=earnings&time_to={time_to}&limit=1000&apikey={api_key}"
    r = requests.get(url)
    data = r.json()
    return data

def _get_label_sentiment(x):
    if x <= -0.35:
        return "Bearish","Bearish"
    elif -0.35 < x <= -0.15:
        return "Somewhat-Bearish","Bearish"
    elif -0.15 < x < 0.15:
        return "Neutral","Neutral"
    elif 0.15 <= x < 0.35:
        return "Somewhat_Bullish","Bullish"
    else:  # x >= 0.35
        return "Bullish","Bullish"
    

_get_data("GOOG","20200101T0000","20220303T0800","YHJOJUT5YAKRNBR5")

In [ ]:
#Youtube video
#https://www.youtube.com/watch?v=ascf3y7zSaY


def get_dataset(time_from="20200101T0000",
                time_to="",
                MAX_API_CALLS_PER_DAY = 25, # Free tier only allows 25 API calls per day
                MAX_API_CALLS_PER_MIN = 5 # Free tier only allows 5 api calls per minute
               ):
    data_list=[]
    for i in range(1,MAX_API_CALLS_PER_DAY+1): 
        if i%5==0: 
            time.sleep(60)
        
        data=_get_data("GOOG",time_from,time_to,api_key)
        print(data)
        if "feed" not in data:break
        if len(data["feed"])==0: break
        data_list.append(data)
        time_to=data["feed"][-1]["time_published"][:-2] # Take all the way up to last 2 since api only takes minute level granularity
    df=pd.concat([pd.DataFrame(data["feed"]) for data in data_list])
    # Extract TSLA specific relevance (we didn"t use it in video)
    df["ticker_relevance_GOOG"]=df["ticker_sentiment"].apply(lambda l:[el for el in l if el["ticker"]=="GOOG"][0]["relevance_score"]).astype(float)
    # Extract TSLA specific sentiment
    df["ticker_sentiment_GOOG"]=df["ticker_sentiment"].apply(lambda l:[el for el in l if el["ticker"]=="GOOG"][0]["ticker_sentiment_score"]).astype(float)
    # Only take tickers with TSLA in headline 
    #df=df[df.title.str.contains("tsla|tesla",case=False)]
    # Extract # of tickers
    df["num_tickers"]=df.ticker_sentiment.apply(lambda l:len(l))
    # Only take when # of tickers = 1
    #df = df[df.num_tickers==1]
    # Applying the function and creating two new columns
    df[["detailed_original_label","label"]] = df.apply(lambda row: _get_label_sentiment(row["ticker_sentiment_GOOG"]), axis=1, result_type="expand")
    # Drop duplicates..
    df.drop_duplicates(subset=["summary"],inplace=True,keep="first")
    # Set index to time published
    df.set_index("time_published",inplace=True)
    # Sort by time published
    df.sort_index(inplace=True)
    return df

In [ ]:
df = get_dataset(time_to="20220303T0800")
df.to_csv("GOOG_sentiment1000_earnings_20220303T08.csv")

In [ ]:
df = pd.read_csv(r"C:\Users\matti\OneDrive\Thesis\Data\GOOG_sentiment1000_earnings.csv",index_col="time_published").sort_index()
earliest_time_published = df.index[0]
earliest_time_published

In [ ]:
df_new = get_dataset(time_to=earliest_time_published[:-2])

In [ ]:
df = pd.read_csv(r"C:\Users\matti\OneDrive\Thesis\Data\GOOG_sentiment1000_earnings.csv", header=None)

df.columns = [
    "timestamp", "title", "url", "authors", "summary", "image_url",
    "source", "category", "domain", "topics", "overall_sentiment_score",
    "overall_sentiment_label", "tickers", "max_relevance_score",
    "max_sentiment_score", "ticker_count", "min_sentiment_label", "max_sentiment_label"
]

df["date"] = df["timestamp"].str[:8]

def extract_goog_sentiment(ticker_data):
    try:
        tickers = ast.literal_eval(ticker_data)
        for ticker in tickers:
            if ticker["ticker"] == "GOOG":
                return float(ticker["ticker_sentiment_score"])
    except (ValueError, SyntaxError):
        return None
    return None

df["GOOG_sentiment"] = df["tickers"].apply(extract_goog_sentiment)

article_counts = df.groupby("date").size()

goog_sentiments = df["GOOG_sentiment"].dropna()
mean_sentiment = goog_sentiments.mean()
std_sentiment = goog_sentiments.std()
range_sentiment = goog_sentiments.max() - goog_sentiments.min()
max_sentiment = goog_sentiments.max()
min_sentiment = goog_sentiments.min()
median_sentiment = goog_sentiments.median()
iqr_sentiment = np.percentile(goog_sentiments, 75) - np.percentile(goog_sentiments, 25)

print("Article counts per date:")
print(article_counts)
print("\nGOOG Sentiment Stats:")
print(f"Mean: {mean_sentiment:.4f}")
print(f"Standard Deviation: {std_sentiment:.4f}")
print(f"IQR: {iqr_sentiment:.4f}")
print(f"Median: {median_sentiment:.4f}")
print(f"Range: {range_sentiment:.4f}")
print(f"Max: {max_sentiment:.4f}")
print(f"Min: {min_sentiment:.4f}")


In [ ]:
file1 = r"C:\Users\matti\OneDrive\Thesis\Data\GOOG_sentiment1000_earnings.csv"
file2 = r"C:\Users\matti\OneDrive\Thesis\Data\GOOG_sentiment1000_earnings_20230213T2040.csv"


df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

#Combine with df1 so that df2"s oldest entries are first
combined_df = pd.concat([df2, df1], ignore_index=True)


#Save to a new file
combined_df.to_csv("aggregated_news_data.csv", index=False)

In [ ]:
combined_df = pd.read_csv(r"C:\Users\matti\OneDrive\Thesis\Data\aggregated_news_data.csv")
#print(combined_df.columns)

final = combined_df[["time_published", "title", "summary", "ticker_relevance_GOOG", "ticker_sentiment_GOOG"]]

final.to_csv("filtered_sentiment_data.csv", index=False)

In [ ]:
df = pd.read_csv(r"C:\Users\matti\OneDrive\Thesis\Data\filtered_sentiment_data.csv")

df["time_published"] = pd.to_datetime(df["time_published"], format="%Y%m%dT%H%M%S")
df["time_published"] = df["time_published"].dt.strftime("%Y-%m-%d")

duplicates_title = df[df.duplicated(subset=["title","time_published"], keep=False)]
print(len(duplicates_title))

print(duplicates_title[["time_published", "title"]].sort_values("title"))
print()

#Checked a few and seems like the titles are re-used, but the articles are different
#Authors are usually the same, with exception of a few, but topics, relevance scores and sentiment scores are different
duplicates = df[df.duplicated(subset="title", keep=False)]
print(len(duplicates))

print(duplicates[["time_published", "title"]].sort_values("title"))


df_cleaned = df.drop_duplicates(subset=["title", "time_published"], keep="first")
df_cleaned.to_csv(r"C:\Users\matti\OneDrive\Thesis\Data\filtered_sentiment_data_no_duplicates.csv", index=False)


In [ ]:
df = pd.read_csv(r"C:\Users\matti\OneDrive\Thesis\Data\filtered_sentiment_data_no_duplicates.csv")

#Calculate weighted sum
df["time_published"] = pd.to_datetime(df["time_published"])
df["date"] = df["time_published"].dt.date

daily_sentiment = (
    df.groupby("date")
    .apply(lambda g: (g["ticker_sentiment_GOOG"] * g["ticker_relevance_GOOG"]).sum() / g["ticker_relevance_GOOG"].sum())
    .reset_index(name="daily_weighted_sentiment_GOOG"))

daily_sentiment.to_csv("F_daily_sentiment_GOOG.csv", index=False)

In [ ]:
#check for missing dates
df = pd.read_csv(r"C:\Users\matti\OneDrive\Thesis\Data\F_daily_sentiment_GOOG.csv", parse_dates=["date"])

df.set_index("date", inplace=True)

start_date = "2022-03-03"
end_date = "2024-12-31"

full_range = pd.date_range(start=start_date, end=end_date)
missing_dates = full_range.difference(df.index)
print(missing_dates)

#Missing 2022-03-27 since the very start

In [ ]:
#Fix missing date with forward filling

trading_idx = pd.date_range(start=start_date, end=end_date, freq='B')  #'B' = business days
fwd_1d = df.reindex(trading_idx, method="ffill", limit=3)
added_rows = fwd_1d.index.difference(df.index)

common_idx = df.index.intersection(fwd_1d.index)
original_unchanged = fwd_1d.loc[common_idx].equals(df.loc[common_idx])
print("Original data unchanged:", original_unchanged)

fwd_1d.index.name = "date"

fwd_1d.to_csv("FF_daily_sentiment_GOOG.csv", index=True)

#Fixed

In [ ]:
#Create folling sentiment features, as it could be relevant to know not just yesterday's sentimement but further past
df = pd.read_csv(r"C:\Users\matti\OneDrive\Thesis\Data\FF_daily_sentiment_GOOG.csv", parse_dates=["date"])
df.set_index("date", inplace=True)

#Merge with rest of data
sent = df["daily_weighted_sentiment_GOOG"]

roll_3d = sent.rolling(3).mean().reindex(trading_idx, method="ffill", limit=3).to_frame(name="sent_roll_3d")
roll_5d = sent.rolling(5).mean().reindex(trading_idx, method="ffill", limit=5).to_frame(name="sent_roll_5d")
roll_7d = sent.rolling(7).mean().reindex(trading_idx, method="ffill", limit=7).to_frame(name="sent_roll_7d")

sent_features = pd.concat([fwd_1d, roll_3d, roll_5d, roll_7d], axis=1)

#Merge with price and indicator data
sent_features.to_csv("FINAL_daily_sentiment_GOOG.csv", index=True)

